In [1]:
import pandas as pd

df = pd.read_csv('/Users/sa22/Documents/code/Price_Prophet/data/raw/train.csv')

In [2]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [3]:
processedDF = df.drop(columns=['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'GarageCars', 'TotRmsAbvGrd', 'GarageYrBlt', 'SalePrice'])

Dropping:

Id because it is meaningless to any analysis. 

PoolQC, MiscFeature, Alley, Fence, and MasVnrType for the large amount of Nans.

GarageCars, TotRmsAbvGrd, and GarageYrBlt for the high collinearity with GarageArea, GrLivArea, and YearBuilt respectively.

SalePrice because it is the target variable.

# One-Hot Encoding the categorical values

In [4]:
cat_cols = processedDF.select_dtypes(include=["object", "category"]).columns.tolist()

df_encoded = pd.get_dummies(processedDF, columns=cat_cols, drop_first=True)

df_encoded.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,False,False,False,False,True,False,False,False,True,False
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,False,False,False,False,True,False,False,False,True,False
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,False,False,False,False,True,False,False,False,True,False
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,False,False,False,False,True,False,False,False,False,False
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,False,False,False,False,True,False,False,False,True,False


In [5]:
# converts remaining bools into numbers

bool_cols = df_encoded.select_dtypes(include='bool').columns

df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)


In [6]:
df_encoded.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,1,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,1,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,1,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,1,0,0,0,1,0


# Checking for nulls and filling missing values with the median

In [7]:
df_encoded.isna().sum().sort_values(ascending=False)

LotFrontage              259
MasVnrArea                 8
BsmtCond_Po                0
ExterCond_Gd               0
ExterCond_Po               0
                        ... 
Condition1_RRAn            0
Condition1_RRNe            0
Condition1_RRNn            0
Condition2_Feedr           0
SaleCondition_Partial      0
Length: 230, dtype: int64

In [8]:
# Impute missing values with the median of the column
df_encoded['LotFrontage'] = df_encoded['LotFrontage'].fillna(df_encoded['LotFrontage'].median())
df_encoded['MasVnrArea'] = df_encoded['MasVnrArea'].fillna(df_encoded['MasVnrArea'].median())

In [9]:
df_encoded.isna().sum().sort_values(ascending=False)

MSSubClass               0
ExterQual_TA             0
ExterCond_Gd             0
ExterCond_Po             0
ExterCond_TA             0
                        ..
Condition1_RRAn          0
Condition1_RRNe          0
Condition1_RRNn          0
Condition2_Feedr         0
SaleCondition_Partial    0
Length: 230, dtype: int64

In [10]:
df_encoded.to_csv('/Users/sa22/Documents/code/Price_Prophet/data/filtered/processedData.csv')